In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [2]:
# Load the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_data = torchvision.datasets.MNIST(root='/data_torch', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)

test_data = torchvision.datasets.MNIST(root='/data_torch', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=True)

In [3]:
# Define the CNN Model
class ConvNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        # Conv + Pool + Fully Connected + Output
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Convolution layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Fully connected layers
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        # Output 10 classes
        x = self.fc3(x)
        return x

In [4]:
# Train the network
def train(model, trainloader, criterion, optimizer, device):
    model.train()
    loss_epoch = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        loss_epoch += loss.item()
    return loss_epoch / len(trainloader)

In [5]:
# Test the network
def test(model, testloader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

In [6]:
# Define the device to use (GPU/CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# Create the network and move it to the device
net = ConvNet().to(device)

In [8]:
# Define the loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

In [9]:
# Train the CNN
num_epochs = 10

for epoch in range(num_epochs):
    train_loss = train(net, trainloader, criterion, optimizer, device)
    test_acc = test(net, testloader, criterion, device)
    print('Epoch %d: train loss=%.3f, test acc=%.3f'% (epoch, train_loss, test_acc))


Epoch 0: train loss=0.188, test acc=0.982
Epoch 1: train loss=0.073, test acc=0.984
Epoch 2: train loss=0.069, test acc=0.985
Epoch 3: train loss=0.065, test acc=0.986
Epoch 4: train loss=0.065, test acc=0.985
Epoch 5: train loss=0.059, test acc=0.983
Epoch 6: train loss=0.064, test acc=0.985
Epoch 7: train loss=0.055, test acc=0.984
Epoch 8: train loss=0.058, test acc=0.984
Epoch 9: train loss=0.060, test acc=0.983


: 